앙상블 모형 개선4.1, K-Fold 사용 (GPT 코드 복붙)  
평균 mae = 1.1958

In [ ]:
# 1. 필요한 라이브러리 설치
!pip install pandas numpy scikit-learn matplotlib seaborn

# 2. 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor

# 3. 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
submission_df = pd.read_csv("data/sample_submission.csv")

# 원본 ID 저장
test_ids = test_df["id"].copy()
test_df = test_df.drop(columns=["id"])  # 분석을 위해 ID 삭제

# 4. 데이터 전처리
# 성별(Label Encoding)
label_encoder = LabelEncoder()
train_df["Sex"] = label_encoder.fit_transform(train_df["Sex"])
test_df["Sex"] = label_encoder.transform(test_df["Sex"])

# 키(Height) 0 값 평균으로 대체
height_mean = train_df.loc[train_df["Height"] > 0, "Height"].mean()
train_df.loc[train_df["Height"] == 0, "Height"] = height_mean
test_df.loc[test_df["Height"] == 0, "Height"] = height_mean

# IQR 기반 이상치 제거
def remove_outliers_iqr(df, cols, threshold=1.5):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

num_cols = train_df.select_dtypes(include=["float64"]).columns
train_df = remove_outliers_iqr(train_df, num_cols, threshold=1.5)

# 중복 데이터 제거 및 ID 삭제
train_df = train_df.drop_duplicates().drop(columns=["id"])

# X, y 분리
X = train_df.drop(columns=["Age"])
y = train_df["Age"]

# 5. K-Fold 설정 (5-Fold 교차 검증)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []  # MAE 저장 리스트

# 반복문으로 K-Fold 실행 (5회)
for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"\n🚀 Fold {fold+1} 시작...")

    # 데이터 분할
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    # 6. 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # 7. PCA 적용 (95% 이상의 분산 유지)
    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_valid_pca = pca.transform(X_valid_scaled)

    # 8. 하이퍼파라미터 튜닝
    # 랜덤 포레스트 최적 하이퍼파라미터 찾기
    rf_param_grid = {
        "n_estimators": [50, 100, 200],
        "max_depth": [5, 10, 20],
        "min_samples_split": [5, 10],
        "max_features": [0.5, "sqrt", "log2"]
    }
    rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, 
                                  scoring="neg_mean_absolute_error", cv=5, n_jobs=-1)
    rf_grid_search.fit(X_train_pca, y_train)
    rf_best_model = RandomForestRegressor(**rf_grid_search.best_params_, random_state=42)

    # 그래디언트 부스팅 최적 하이퍼파라미터 찾기
    gb_param_grid = {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3, 5, 7],
        "subsample": [0.7, 0.8, 0.9]
    }
    gb_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_param_grid, 
                                  scoring="neg_mean_absolute_error", cv=5, n_jobs=-1)
    gb_grid_search.fit(X_train_pca, y_train)
    gb_best_model = GradientBoostingRegressor(**gb_grid_search.best_params_, random_state=42)

    # 9. 앙상블 모델 (Voting Regressor)
    lr_model = LinearRegression()
    ensemble_model = VotingRegressor(estimators=[
        ('lr', lr_model),
        ('rf', rf_best_model),
        ('gb', gb_best_model)
    ], weights=[0.5, 1, 2])

    # 10. 모델 학습
    ensemble_model.fit(X_train_pca, y_train)

    # 11. 검증 데이터 예측 및 평가
    y_pred = ensemble_model.predict(X_valid_pca)
    fold_mae = mean_absolute_error(y_valid, y_pred)
    mae_scores.append(fold_mae)

    print(f"✅ Fold {fold+1} MAE: {fold_mae:.4f}")

# 12. K-Fold 평균 MAE 출력
mean_mae = np.mean(mae_scores)
print(f"\n🔥 K-Fold Cross Validation 평균 MAE: {mean_mae:.4f}")

# 13. 전체 데이터로 최종 학습 후 테스트 예측
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
test_scaled = scaler.transform(test_df)
test_pca = pca.transform(test_scaled)

# 최적 모델로 최종 학습
rf_grid_search.fit(X_pca, y)
rf_best_model = RandomForestRegressor(**rf_grid_search.best_params_, random_state=42)

gb_grid_search.fit(X_pca, y)
gb_best_model = GradientBoostingRegressor(**gb_grid_search.best_params_, random_state=42)

ensemble_model = VotingRegressor(estimators=[
    ('lr', LinearRegression()),
    ('rf', rf_best_model),
    ('gb', gb_best_model)
], weights=[0.5, 1, 2])

ensemble_model.fit(X_pca, y)

# 14. 테스트 데이터 예측
test_preds = ensemble_model.predict(test_pca)

# 15. 제출 파일 생성
submission = pd.DataFrame({"id": test_ids, "Age": np.round(test_preds, 3)})
submission.to_csv("download/sample_submission.csv", index=False)
print("✅ sample_submission.csv 파일 생성 완료!")



🚀 Fold 1 시작...
✅ Fold 1 MAE: 1.2014

🚀 Fold 2 시작...
✅ Fold 2 MAE: 1.2136

🚀 Fold 3 시작...
✅ Fold 3 MAE: 1.1694

🚀 Fold 4 시작...
✅ Fold 4 MAE: 1.1951

🚀 Fold 5 시작...
✅ Fold 5 MAE: 1.1994

🔥 K-Fold Cross Validation 평균 MAE: 1.1958
✅ sample_submission.csv 파일 생성 완료!


### 📌 코드 설명  
✅ 패키지 설치: 필요한 라이브러리를 설치하고 import  
✅ 데이터 로드 및 전처리: 이상치 제거, 라벨 인코딩, 스케일링  
✅ K-Fold 적용: 5개의 Fold로 데이터를 나누어 모델을 평가  
✅ GridSearchCV 하이퍼파라미터 튜닝: RandomForest와 GradientBoosting 최적화  
✅ Voting Regressor 적용: 세 가지 모델을 조합하여 성능 향상  
✅ 최종 예측 및 제출 파일 생성: 최적 모델로 test.csv 예측 후 CSV 저장  

---

### 💡 K-Fold를 사용하면...  
✔ 모든 데이터를 학습에 활용 → 일반적인 train_test_split()보다 성능이 안정적  
✔ 과적합 방지 → 여러 번 평가하여 최적의 모델 선택  
✔ 더 정확한 성능 측정 → 여러 번 학습하고 MAE 평균을 계산  

이제 K-Fold를 사용한 모델로 나이 예측을 더 정확하게 할 수 있을 거야! 🚀🔥